In [ ]:
# merged_email_agent.py
import os
import json
from dotenv import load_dotenv
import pypdf

# LangChain imports
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.memory import ConversationBufferMemory
from langchain_community.utilities import SerpAPIWrapper
from langchain.agents import initialize_agent, AgentType

try:
    from langchain.tools import Tool
except Exception:
    from langchain_core.tools import Tool

# === Load environment variables ===
load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
SERPAPI_KEY = os.getenv("SERPAPI_API_KEY") or os.getenv("SERP_API_KEY")

if not GOOGLE_API_KEY:
    raise RuntimeError("GOOGLE_API_KEY not set in .env")
if not SERPAPI_KEY:
    raise RuntimeError("SERPAPI_API_KEY (or SERP_API_KEY) not set in .env")

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

# === PDF Resume Extraction ===
def extract_text_from_pdf(file_path):
    with open(file_path, "rb") as pdf_file:
        reader = pypdf.PdfReader(pdf_file)
        text = ""
        for page in reader.pages:
            text += page.extract_text() or ""
    return text.strip()

# === Search Tool ===
search = SerpAPIWrapper(serpapi_api_key=SERPAPI_KEY, params={"engine": "google"})
search_tool = Tool(
    name="Search",
    func=search.run,
    description="Run a web search (SerpAPI) for job descriptions and company info."
)

# === Memory & LLM ===
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.7,
    model_kwargs={
        "instructions": (
            "You are a career assistant that analyzes resumes against job descriptions, "
            "identifies improvements or mistakes, and writes professional recruiter emails. "
            "If the resume is well-suited, respond with 'All good' in the improvement section. "
            "Use SerpAPI search for extra company details if needed."
        )
    }
)

# === Agent Setup ===
agent = initialize_agent(
    tools=[search_tool],
    llm=llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

# === Main Execution ===
if __name__ == "__main__":
    # === Step 1: Inputs ===
    pdf_path = input("Enter path to your resume PDF: ").strip()
    job_description = input("Paste the job description here: ").strip()

    resume_text = extract_text_from_pdf(pdf_path)

    # === Step 2: Agent Task ===
    task_prompt = f"""
    Here is the job description:
    {job_description}

    Here is my resume content:
    {resume_text}

    1. Analyze the resume against the job description for missing skills, errors, or possible improvements.
    2  Tell us where resume can be improved or if it matches the job description.
    3. mention the keywords from the job description if that are missing in the resume.
    4. make sure resume should pass 70 percent match with the job description.
    5. If everything matches well, say "All good" in the improvement section.
    6. Then, write a professional, concise email to the hiring manager highlighting my relevant skills.
    """

    response = agent.invoke({"input": task_prompt})

    
    # === Step 4: Output ===
    print("\n=== Agent Output ===")
    print(response)

